<H1>Create rigid constraints (links) between deck beams and pier legs</H1>

In [ ]:
# Input Params
spans_lengths = [5, 10, 5] #m
tee_section_width = 1.0
tee_section_depth = 1.0
tee_section_thk_flange = 0.2
tee_section_thk_web = 0.2

loading = 10

In [ ]:
import win32com.client as win32
lusas = win32.gencache.EnsureDispatch("Lusas.Modeller.21.1")

In [ ]:
# Create a new model
db = lusas.newDatabase("Structural", f"RC Beam Design.mdl")
# Set the vertical axis
db.setLogicalUpAxis("Z")
# Set the unit system
db.setModelUnits("kN,m,t,s,C")

lusas.view().setIsometric()

In [ ]:
''' Create deck lines'''
deck_lines = []

cur_x = 0
for length in spans_lengths:
    # Get the geometry data object
    geomData = lusas.geometryData().setAllDefaults()
    # set the options for creating points from coordinates
    geomData.setLowerOrderGeometryType("coordinates")
    # Set the coordinates of the first point
    geomData.addCoords(cur_x, 0, 0)
    # Set the coordinates of the second point
    geomData.addCoords(cur_x + length, 0, 0)
    # Create the line object from the geomety data
    objs = db.createLine(geomData) # Modeller returns an object set containing all the lines created, in this case we only have 1.
    # Get the line
    deck_lines.append(objs.getObjects("Lines")[0])
    # Increment the current x position
    cur_x += length


In [ ]:
''' Create a mesh attribute'''
# Beam mesh with BMI21 (2 Noded Linear) Elements at 1m lengths
mesh_attr = db.createMeshLine("Beam Mesh").setSize("BMI21", 1)

In [ ]:
''' Create a geometric attribute'''

# Create a parametric section utility
util = db.createParametricSection('Tee Section Utility')
util.setType("Reactangular Solid")
util.setDimensions(['B', 'D', 'tf', 'tw'], [tee_section_width, tee_section_depth, tee_section_thk_flange, tee_section_thk_web])

# Now create the attribute which is based on the utility but provides additional transformations such as eccentrity and tapering
geom_section_attr = db.createGeometricLine('Tee Section')
geom_section_attr.setFromLibrary("Utilities", "", 'Tee Section Utility', 0, 0, 0)


In [ ]:
''' Create a material attribute'''
concrete_material_attr = db.createIsotropicMaterial("Concrete", 35_000_000, 0.2, 2.4, 10e-6)
rebar_material_attr = db.createIsotropicMaterial("Rebar", 209_000_000, 0.3, 7.8, 10e-6)

lusas.getModule("MaterialLibrary").createMaterial("Europe", "EN1992-1-1:2005")


rc_material_attr = None

In [ ]:
''' Create a support attribute '''
fixedSupport = db.createSupportStructural("Fully Fixed")
# set the freedoms F=Free, R=Restrained
fixedSupport.setStructural("R", "R", "R", "R", "R", "R", "F", "F", "F")

''' Create a support attribute '''
pinnedSupport = db.createSupportStructural("Pinned")
# set the freedoms 
pinnedSupport.setStructural("F", "R", "R", "F", "F", "F", "F", "F", "F")

In [ ]:
''' Create a beam load attribute '''
loadAttr = db.createLoadingBeamDistributed("UDL")
loadAttr.setBeamDistributed("Parametric", "Global", "beam")
loadAttr.addRow(0.0, 0.0, 0.0, -loading, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, -loading, 0.0, 0.0, 0.0)

In [ ]:
''' Assign the attributes to the deck_lines '''
# get the assignment object
assignment = lusas.assignment().setAllDefaults()
# Assign the mesh
mesh_attr.assignTo(deck_lines, assignment)
# Assign the geometry
geom_section_attr.assignTo(deck_lines, assignment)
# Assign the material
rc_material_attr.assignTo(deck_lines, assignment)
# Assign the loading
loadAttr.assignTo(deck_lines, assignment)

''' Assign the supports to the points of the line '''
# Assign the pinned support to the first point
pinnedSupport.assignTo(deck_lines[0].getStartPoint(), assignment)
# Assign the pinned support to the last point
pinnedSupport.assignTo(deck_lines[-1].getEndPoint(), assignment)

In [ ]:
db.updateMesh()

<H2>Solving the Analysis</H2>

In [ ]:
db.getAnalysis("Analysis 1").solve(True)
db.openAllResults(False)

In [ ]:
lusas.view().insertDiagramsLayer()
lusas.view().diagrams().setResultsTransformNone()
lusas.view().diagrams().setResults("Force/Moment - Thick 3D Beam", "My")
